<div align="center">
<img src="https://poorit.in/image.png" alt="Poorit" width="40" style="vertical-align: middle;"> <b>AI SYSTEMS ENGINEERING 1</b>

## Unit 2: Conversational AI with Memory

**CV Raman Global University, Bhubaneswar**  
*AI Center of Excellence*

---

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Poorit-Technologies/cvraman-coe/blob/main/courses-contents/ai-systems-engineering-1/unit-2/02-ai-systems-engineering-1-unit2-chat-with-memory.ipynb)

</div>

---

### What You'll Learn

In this notebook, you will:

1. **Build a chatbot with conversation memory** using Gradio's ChatInterface
2. **Understand how history is managed** in LLM conversations
3. **Use system prompts for context** to customize chatbot behavior
4. **Implement streaming chat responses** for better UX

**Duration:** ~1 hour

---

## 1. Environment Setup

In [ ]:
# Install required packages
!pip install -q openai gradio

In [ ]:
import os
from getpass import getpass
from openai import OpenAI
import gradio as gr

In [ ]:
# Configure OpenAI
api_key = getpass("Enter your OpenAI API Key: ")
os.environ['OPENAI_API_KEY'] = api_key
client = OpenAI(api_key=api_key)
MODEL = "gpt-4o-mini"

---

## 2. Understanding ChatInterface

Gradio's `ChatInterface` is specifically designed for chatbot UIs.

It expects a callback function with this signature:
```python
def chat(message, history):
    # message: the user's current message
    # history: list of previous messages
    return response
```

In [ ]:
# Simple chatbot - just echoes
def chat(message, history):
    return f"You said: {message}"

gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
# Let's see what history looks like
def chat(message, history):
    return f"Message: {message}\n\nHistory: {history}"

gr.ChatInterface(fn=chat, type="messages").launch()

---

## 3. Connecting to an LLM with Memory

Now let's connect to a real LLM and include the conversation history.

In [ ]:
system_message = "You are a helpful assistant."

def chat(message, history):
    # Convert Gradio history format to OpenAI format
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    
    # Build the messages list
    messages = (
        [{"role": "system", "content": system_message}]
        + history
        + [{"role": "user", "content": message}]
    )
    
    response = client.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

---

## 4. Adding Streaming

For a better user experience, let's stream the response.

In [ ]:
def chat(message, history):
    # Convert history format
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    
    # Build messages
    messages = (
        [{"role": "system", "content": system_message}]
        + history
        + [{"role": "user", "content": message}]
    )
    
    # Stream the response
    stream = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        stream=True
    )
    
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response

gr.ChatInterface(fn=chat, type="messages").launch()

---

## 5. Customizing Behavior with System Prompts

System prompts allow you to customize the chatbot's personality and behavior.

Let's create a shopping assistant for a clothing store.

In [ ]:
system_message = """
You are a helpful assistant in a clothing store called FashionHub.
You should gently encourage customers to try items that are on sale.
Kurtas are 60% off, and most other items are 50% off.

For example, if a customer says 'I'm looking for traditional wear',
you could reply: 'Wonderful! We have beautiful kurtas that are part of our sale - 60% off!'

Be friendly and helpful. Keep responses concise.
"""

gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
# Let's add more context
system_message += """
If the customer asks for shoes, mention that shoes are not on sale today,
but suggest they check out our kurtas and other traditional wear!
"""

gr.ChatInterface(fn=chat, type="messages").launch()

---

## 6. Dynamic System Prompts

We can modify the system prompt based on the user's message for more context-aware responses.

In [ ]:
base_system_message = """
You are a helpful assistant in a clothing store called FashionHub.
You should gently encourage customers to try items that are on sale.
Kurtas are 60% off, and most other items are 50% off.
Be friendly and helpful. Keep responses concise.
"""

def chat(message, history):
    # Convert history format
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    
    # Dynamic system prompt based on message content
    current_system = base_system_message
    
    if "belt" in message.lower():
        current_system += " The store does not sell belts. Suggest kurtas or other items instead."
    
    if "return" in message.lower() or "refund" in message.lower():
        current_system += " For returns/refunds, direct customers to visit the customer service desk with their receipt."
    
    # Build messages
    messages = (
        [{"role": "system", "content": current_system}]
        + history
        + [{"role": "user", "content": message}]
    )
    
    # Stream response
    stream = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        stream=True
    )
    
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response

gr.ChatInterface(fn=chat, type="messages").launch()

---

## 7. Exercise: Build a Customer Support Chatbot

Create a chatbot for an Indian airline that:
1. Answers questions about flights and bookings
2. Provides information about baggage policies
3. Suggests premium services when appropriate

In [ ]:
# Exercise: Create an airline customer support chatbot
# Define a system prompt with relevant context
# Implement the chat function with streaming

# Your implementation here
pass

---

## Key Takeaways

1. **ChatInterface simplifies chatbot UIs** - handles history automatically

2. **History management is crucial** - convert formats between Gradio and OpenAI

3. **System prompts define behavior** - use them for personality, context, and rules

4. **Dynamic prompts add flexibility** - modify system messages based on context

### Message Structure Recap

```python
messages = [
    {"role": "system", "content": "You are..."},
    {"role": "user", "content": "First user message"},
    {"role": "assistant", "content": "First response"},
    {"role": "user", "content": "Second user message"},
    # ... and so on
]
```

### What's Next?

In the next notebook, we'll explore:
- Tool calling (function calling)
- Giving LLMs the ability to execute actions
- Database integration

---

## Additional Resources

- [Gradio ChatInterface](https://www.gradio.app/docs/chatinterface)
- [OpenAI Chat Completions](https://platform.openai.com/docs/guides/text-generation)

---

**Course Information:**
- **Institution:** CV Raman Global University, Bhubaneswar
- **Program:** AI Center of Excellence
- **Course:** AI Systems Engineering 1
- **Developed by:** [Poorit Technologies](https://poorit.in) - *Transform Graduates into Industry-Ready Professionals*

---